In [1]:
import pandas as pd
import os
from os import path
import numpy as np

In [2]:
# create a list of the dataSets directory content
listdir = os.listdir('dataSets')

In [3]:
# # and check if there are subdirectories in it 
# if (path.exists(f'dataSets\\{listdir[0]}')):
#         listSubDir = os.listdir(f'dataSets\\{listdir[0]}')

In [4]:
# #extract each csv, convert to a DataFrame and concatenate them
# if (path.exists(f'dataSets')):
#     # create a list of subdirectories
#     listdir = os.listdir('dataSets')
#     # iterate over the list of subdirectories
#     for i in range(len(listdir)):
#         # save the names of the subdirectories
#         pathSub = f'dataSets\\{listdir[i]}'
#         if len(listdir) > 0:
#             listSubDir = os.listdir(f'dataSets\\{listdir[i]}')
#             #iterate over the files 
#             for j in range(len(listSubDir)):
#                 # As we will concatenate all the files of a given directory, we'll need to store the data frames 
#                 # in two different variables to avoid overwriting them.
#                 if j == 0:
#                     df0 = pd.read_csv(f'dataSets\\{listdir[i]}\\{listSubDir[j]}')
#                 else:
#                     df = pd.read_csv(f'dataSets\\{listdir[i]}\\{listSubDir[j]}')
#                     df0 = pd.concat([df0,df])
#             df0.reset_index(drop=True)
#             df0.to_csv(f'{pathSub}\\concatenated.csv', index = False)

In [5]:
#df = pd.read_csv('dataSets\\1930_world_cup\\1930_Argentina.csv')

## Add missing data

First of all, we should add the data of the replacement players, located in the additional tables that we didn't incorporate before because there were only two of them. So we'll input the data manually.

In [6]:
f = open("errors\\additionalTables.txt", "r")
print(f.readline())    

[(1990, 5), (1990, 23)]


The first number of each tuple indicates the year of the World Cup in which they played, and the second one is the table index in the web page HTML code. We can find the tables on: [1990 squads](https://en.wikipedia.org/wiki/1990_FIFA_World_Cup_squads). Then, look for the 6th and 24rd tables, they should have only one row.

In [15]:
arg = pd.read_csv('dataSets\\1990_world_cup\\1990_Argentina.csv')

In [16]:
if len(arg) == 22:
    # Create a dictionary with the replacement player's data
    replacementArg = ({
        'Player':["Ángel Comizzo"], 'National Team' :["Argentina"], 'No.':['1'] ,'Pos.':["GK"],
        'Date of birth (age)':["27 April 1962 (aged 28)"], 'Caps':["0"],'Goals':[np.nan], 'Club':["River Plate"],
        'Club Origin':["Argentina"],'World Cup start date':["1990-06-08"]
    })
    
    # then, create a data frame from the dictionary
    dfRArg = pd.DataFrame(replacementArg)
    # lastly, concatenate the data frames
    arg = pd.concat([arg,dfRArg], ignore_index = True)
    arg.to_csv('dataSets\\1990_world_cup\\1990_Argentina.csv', index = False)

In [9]:
eng = pd.read_csv('dataSets\\1990_world_cup\\1990_England.csv')

In [10]:
if len(eng) == 22:
    # Create a dictionary with the replacement player's data
    replacementEng = ({
        'Player':["Dave Beasant"], 'National Team' :["England"], 'No.':['22'], 'Pos.':["GK"],
        'Date of birth (age)':["20 March 1959 (aged 31)"], 'Caps':["2"], 'Goals':[np.nan], 'Club':["Chelsea"],
        'Club Origin':["England"],'World Cup start date':["1990-06-08"]
    })
    # create the data frame from the dictionary
    dfREng = pd.DataFrame(replacementEng)
    # concatenate the data frames
    eng = pd.concat([eng,dfREng], ignore_index = True)
    eng.to_csv('dataSets\\1990_world_cup\\1990_England.csv', index = False)

In [ ]:
#arg = arg.drop(columns = ["Unnamed: 0"])

In [ ]:
#arg.to_csv('dataSets\\1990_world_cup\\1990_Argentina.csv', index = False)

In [ ]:
# create the data frame from the dictionary
#dfREng = pd.DataFrame(replacementEng)

Let's load the corresponding national teams data frames

In [ ]:
#eng = eng.drop(columns=["Unnamed: 0"])

## Load data to a sql database

Here we will use sqlite only as a tool to append all the squads data frames. This way seems to be more effective than concatenating the dataframes with pandas.

#### import a database manager

In [18]:
import sqlite3

In [19]:
# make a new directory to locate the database
os.makedirs('dataBases', exist_ok=True) 

In [20]:
# create the data base file
db = 'dataBases//world_cups_squads.db'

#### Feed the data into the database.

In [21]:
# The function below creates a database or connects to an existing one if it already exists, then makes a table and storages
# the data.

In [22]:
def dataset_to_table(df,database):
    conn = sqlite3.connect(database)
    c = conn.cursor()
    
    c.execute("""CREATE TABLE IF NOT EXISTS squads (player TEXT, national_team TEXT, shirt_number INTEGER,
            position TEXT, birthday TEXT, caps INTEGER, goals INTEGER, club TEXT, club_origin TEXT, 
            wc_date TEXT)""")
    
    conn.commit()
    
    df.to_sql('squads', conn, if_exists='append', index = False)
    # df.to_sql('squads', conn, if_exists= 'replace', , index = False)  # Just in case you want to start over
    conn.close

In [23]:
#extract each csv, convert to a DataFrame and concatenate them
if (path.exists(f'dataSets')):
    # create a list of subdirectories
    listdir = os.listdir('dataSets')
    # iterate over the list of subdirectories
    for i in range(len(listdir)):
        # save the names of the subdirectories
        pathSub = f'dataSets\\{listdir[i]}'
        if len(listdir) > 0:
            listSubDir = os.listdir(f'dataSets\\{listdir[i]}')
            #iterate over the files 
            for j in range(len(listSubDir)):
                    df0 = pd.read_csv(f'dataSets\\{listdir[i]}\\{listSubDir[j]}')
                    
                    df0 = df0.rename(columns={'Player': 'player', 'National Team': 'national_team' ,'No.':'shirt_number' ,
                                              'Pos.' :'position','Date of birth (age)' : 'birthday', 'Caps': 'caps' ,
                                              'Goals' :'goals','Club': 'club','Club Origin': 'club_origin',
                                              'World Cup start date' : 'wc_date'})
                    
                    df0.reset_index(drop=True)
                    dataset_to_table(df0, db)

#### Finally,  make sure that the data is available, to do this we load the table and turn it back to a dataframe.¶

In [24]:
def load_table_as_df(database):
    conn = sqlite3.connect(database)
    df = pd.read_sql_query("SELECT * FROM squads", conn)
    conn.close()
    return df

In [25]:
df10 = load_table_as_df(db)

In [28]:
df10.loc[4700:4720]

,player,national_team,shirt_number,position,birthday,caps,goals,club,club_origin,wc_date
4700,Néstor Fabbri,Argentina,11,DF,(1968-04-29),7,NaN,Racing Club,Argentina,1990-06-08
4701,Sergio Goycochea,Argentina,12,GK,(1963-10-17),1,NaN,Millonarios,Colombia,1990-06-08
4702,Néstor Lorenzo,Argentina,13,DF,(1966-02-28),12,NaN,Bari,Italy,1990-06-08
4703,Ricardo Giusti,Argentina,14,MF,(1956-12-11),47,NaN,Independiente,Argentina,1990-06-08
4704,Pedro Monzón,Argentina,15,DF,(1962-02-23),10,NaN,Independiente,Argentina,1990-06-08
4705,Julio Olarticoechea,Argentina,16,DF,(1958-10-18),19,NaN,Racing Club,Argentina,1990-06-08
4706,Roberto Sensini,Argentina,17,DF,(1966-10-12),16,NaN,Udinese,Italy,1990-06-08
4707,José Serrizuela,Argentina,18,DF,(1962-06-16),1,NaN,River Plate,Argentina,1990-06-08
4708,Oscar Ruggeri,Argentina,19,DF,(1962-01-26),50,NaN,Real Madrid,Spain,1990-06-08
4709,Juan Simón,Argentina,20,DF,(1960-03-02),3,NaN,Boca Juniors,Argentina,1990-06-08
